# バジルさんの記事「Binanceで数か月運用していた高頻度botの紹介」を実装してみる。
https://note.com/kkngo/n/n13fb59bacc95?sub_rt=share_sb

## TESTNETによる本番モード実行

[バックテスト・フォーワードテストはこちら](./basilbot_backtest.ipynb)

## ロジック
バジルさんのロジックを引用すると以下のとおり

* 取引頻度
    約定履歴から作成した3秒足ごと
* エントリー/イグジット条件
    * 常に買いから入り、終値*0.9995など決め打ちでの買い指値（数字は仮）
    * イグジットも終値*1.0005など決め打ちでの売り指値（数字は仮）

* ピラミッディングやドテンの類はなし
* 手持ちのTUSD（2500TUSDくらい）分のBTCを買って売るだけ
* 4/27に大きく焼かれてからは、直近数時間の値動きが大きすぎるときはエントリーしないように対応
* 約定回数
    200～300回/日


これを今回の実装用に調整・具体化すると以下。

* 3秒毎にエントリーするかどうかを判定。

* エントリー条件
  * 前回のオーダーが残っていない（オーダー中および買いポジション(相当)がない）
  * 直前の値動きが大きくない
    * ３０分足を４本とり、平均の値幅が閾値以下。
* オーダーキャンセル条件
  * EXPIRE_TIME(600秒: 10分)以上約定しないオーダーはキャンセルする。 
* 執行戦略
  * 買い
    * エントリー条件が揃ったら執行
    * 指値: 3秒足の終値 * (1-OFFSET)     // OFFSETは0.0005などの値
    * 数量: 0.001 BTC
  * 売り
    * 買いオーダーの約定が残っていたら執行
    * 指値：買いオーダー発行時の3秒足の終値 * (1+OFFSET)     // OFFSETは0.0005などの値
    * 数量：約定した買い注文と同数（=仮想敵なPosition）
    * その他：売りオーダーがExpireしてCancelされた場合、即、market_orderで投げうる。


# Agent（BOT）の実装

In [ ]:
class BasilAgent:
    def __init__(self):
        self.OFFSET = 0.00_05
        self.EXPIRE_TIME = 600    # 600[sec] = 10[min]
        self.ORDER_SIZE = 0.01
        self.RANGE = 300

    def on_init(self, session):
        session.clock_interval_sec = 3     # 3秒ごとに on_clock を呼び出す        
    
    def on_clock(self, session, clock):
        if session.expire_order(self.EXPIRE_TIME):         # 期限切れの注文をキャンセルする.
            return                                          # 期限切れがあればリターン

        if session.buy_orders or session.sell_orders:   # 既に注文がある場合はリターン
            return

        # 1時間足のレンジを計算してログに出力する。レンジが大きい場合はトレードしない。        
        ohlcv1h = session.ohlcv(60*60, 4)
        range = (ohlcv1h['high']-ohlcv1h['low']).mean()
        session.log_indicator("range", range)

        if self.RANGE < range:
            return

        ohlcv = session.ohlcv(3, 1)         # 3秒足を1本分取得。
        if len(ohlcv) < 1:                 # 3秒間に約定データがない場合リターン
            print("NO OHLCV DATA")
            return

        if session.position <= 0.001:    # ポジションが少ない場合は買い注文を出す。
            order_price = ohlcv['close'][-1] * (1 - self.OFFSET)
            print("BUY ORDER: ", order_price, self.ORDER_SIZE)
            session.limit_order('Buy', order_price, self.ORDER_SIZE - session.position)

        else:                       # ポジションがある場合は売り注文を出す。
            order_price = ohlcv['close'][-1] * (1 + self.OFFSET)
            print("SELL ORDER: ", order_price, session.position)
            session.limit_order('Sell', order_price, session.position)

    def on_update(self, session, updated_order):
        # 売りオーダーが期限切れされた場合には、成り行きで売り注文を出す（ロスカット）
        if updated_order.status == 'Canceled':
            if updated_order.side == 'Sell':
                session.market_order('Sell', updated_order.remaining_size)



In [ ]:
# rbot拡張ライブラリのインストール（２回目は不要です）
! pip install --upgrade pip
! pip install --upgrade rbot

In [ ]:
# 関連ライブラリのインストール
! pip install pyarrow
! pip install polars
! pip install plotly
! pip install nbformat
! pip install numpy
! pip install pandas
! pip install json2html

## RealRun(TESTNET)

テストネットで本番と同じ動作を確認しましょう。

https://testnet.binance.vision

上記URLでログインして（要GitHubID）、APIキーとSECRETを発行します。
その後、環境変数にセットします。

* `BINANCE_API_KEY` APIキーをセットします。
* `BINANCE_API_SECRET`　APIシークレットをセットします。

環境変数設定後は、Jupyuterの再起動が必要になる場合があります。

In [ ]:
from rbot import BinanceConfig, BinanceMarket
from rbot import Runner
from rbot import NOW, DAYS

from json2html import *
from IPython.display import HTML, display

In [ ]:
config = BinanceConfig.TEST_BTCUSDT

# 設定を表示します。きちんとキーが設定されていることを確認してください。（最初の２文字しか表示されません）
HTML(json2html.convert(config.__str__()))

In [ ]:
market = BinanceMarket(config)

BACKTEST_DAYS = 1

market.download(
    ndays=BACKTEST_DAYS,  # 過去１日分のデータをダウンロード
    verbose=True          # ダウンロードの進捗を表示
)

In [ ]:
agent = BasilAgent()        # 実行対象のエージェントを指定
runner = Runner()           # 実行モジュール Runner を作成

In [ ]:
session = runner.real_run(
    agent=agent,
    market=market,
    execute_time=60*60*6,    # 6H      # 実行時間の設定（本番では設定しない。NoteBookでテストするために終了条件を入れている）
    verbose=True,
    log_file='./rbot.log'       # ログファイルを指定
)

## 分析フェーズ

本来BOTは動き続けるので、real_runの戻り値を使うことはできない。そこで、ファイルに出力したログをつかって分析する。Loggerクラスへ復元したあとはこれまで同じやり方が使える。

In [ ]:
from rbot import Logger

log = Logger(True)
log.restore('./rbot.log')


In [ ]:
orders = log.orders

In [ ]:
# オーダー状況分析
# あまりにCancelが多い場合は、Expireの時間やオーダーのOFFSETを調整する必要がある。
orders.group_by(['order_side', 'status']).count()

In [ ]:
# 最終想定損益
orders['sum_profit'][-1]

In [ ]:
# 手数料なし想定損益
orders['profit'].sum()

In [ ]:

# ローソク足データの作成 
ohlcv = market.ohlcv(           
    runner.start_timestamp,     # バックテストの開始時刻
    runner.last_timestamp,    # バックテストの終了時刻
    30                          # 30秒足
)
ohlcv.tail(5)

In [ ]:
# session.log_indicator('range', value)で保存したデータは以下で取得できます。戻り値はpolars.DataFrameです。
log['range'].tail(5)

In [ ]:
# バックテスト結果の可視化

# 数千件のデータを可視化するときは、plotlyを使うと便利。
# https://plotly.com/python/
# ただし約定データが万を超えると、ブラウザが重くなるので注意。一部データ切り取りなどが必要。

import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig = make_subplots(rows=4, cols=1, shared_xaxes=True, vertical_spacing=0.1, row_heights=[0.1, 0.1, 0.1, 0.6],
                    subplot_titles=("range", "psudo-position", "psudo-profit", "candlestick"))

# row 1 (indicator)
spread = log['range']
fig.add_trace(go.Scatter(x=spread['timestamp'], y=spread['range'], name="range"), row=1, col=1)

# row 2 (position)
fig.add_trace(go.Scatter(x=orders['update_time'], y=orders['position'], name="position", line=dict(shape='hv')), row=2, col=1)

# row 3 (profit)
profit = orders['sum_profit']
fig.add_trace(go.Scatter(x=orders['update_time'], y=orders['sum_profit'], name="profit", line=dict(shape='hv')), row=3, col=1)

# row 4 (candlestick)
fig.add_candlestick(x=ohlcv['timestamp'], open=ohlcv['open'], high=ohlcv['high'], low=ohlcv['low'], close=ohlcv['close'], row=4, col=1)

# row 4 (order)
buy_orders = orders.filter((orders['order_side'] == 'Buy') & (orders['status'] == 'New'))
fig.add_trace(go.Scatter(x=buy_orders['update_time'], y=buy_orders['order_price'], mode='markers', marker=dict(symbol='arrow-up', color='red', size=10), name="buy"), row=4, col=1)

buy_orders = orders.filter((orders['order_side'] == 'Buy') & (orders['status'] == 'Filled'))
fig.add_trace(go.Scatter(x=buy_orders['update_time'], y=buy_orders['order_price'], mode='markers', marker=dict(symbol='cross-thin-open', color='red', size=10), name="buy filled"), row=4, col=1)

buy_orders = orders.filter((orders['order_side'] == 'Buy') & (orders['status'] == 'Canceled'))
fig.add_trace(go.Scatter(x=buy_orders['update_time'], y=buy_orders['order_price'], mode='markers', marker=dict(symbol='x-thin-open', color='red', size=10), name="buy canceled"), row=4, col=1)


sell_orders = orders.filter((orders['order_side'] == 'Sell') & (orders['status'] == 'New'))
fig.add_trace(go.Scatter(x=sell_orders['update_time'], y=sell_orders['order_price'], mode='markers', marker=dict(symbol='arrow-down', color='blue', size=10), name="sell"), row=4, col=1)

sell_orders = orders.filter((orders['order_side'] == 'Sell') & (orders['status'] == 'Filled'))
fig.add_trace(go.Scatter(x=sell_orders['update_time'], y=sell_orders['order_price'], mode='markers', marker=dict(symbol='cross-thin-open', color='blue', size=10), name="sell filled"), row=4, col=1)

sell_orders = orders.filter((orders['order_side'] == 'Sell') & (orders['status'] == 'Canceled'))
fig.add_trace(go.Scatter(x=sell_orders['update_time'], y=sell_orders['order_price'], mode='markers', marker=dict(symbol='x-thin-open', color='blue', size=10), name="sell canceled"), row=4, col=1)

fig.update_layout(height=800, title_text="Backtest Result")


# 注意：

APIキーを本番用に変更し、`config`を`BinanceConfig.BTCUSDT`へ変更すると本当にオーダーが発行されます。

ここまでも自己責任ですが、この先も完全自己責任でお楽しみください。